## 1. CSV 데이터 읽어오기(데이터프레임화)

In [1]:
# 1. CSV 데이터 읽어오기(데이터프레임화)

# PANDAS 패키지 불러오기
import pandas as pd
import numpy as np
import math
from isoweek import Week

 
# CSV 파일을 읽어 Data Frame 변수에 저장하기

seasonality_dataData = pd.read_csv("./dataset/seasonality_data.csv")
# pro_promotionData = pd.read_csv("./dataset/pro_promotion.csv")
# pro_actual_salesData = pd.read_csv("./dataset/pro_actual_sales.csv")

# 컬럼해더 재정의
# customerData.columns = ['CUSTID','AVGPRICE','EMI','DEVICECOUNT','PRODUCTAGE','CUSTTYPE']

# DataFrame으로 변환 
seasonality_dataData_df= pd.DataFrame(seasonality_dataData)

# 데이터 VIEW
seasonality_dataData.head()

# CSV 파일로 저장
# customerData.to_csv("../dataset/customerdata_out.csv", encoding='utf-8-sig', index=False)
# customerData.head()

,CONTINENT,REGIONID,PRODUCTGROUP,PRODUCT,YEARWEEK,QTY
0,N05,A60,PG01,PRODUCT04,201402,71.0
1,N05,A60,PG03,PRODUCT59,201402,22275.0
2,N05,A60,PG02,PRODUCT34,201402,4463.0
3,N05,A60,PG03,PRODUCT47,201402,0.0
4,N05,A60,PG03,PRODUCT56,201402,23.0


In [2]:
# 데이터 타입 확인
seasonality_dataData_df.dtypes

CONTINENT        object
REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
YEARWEEK          int64
QTY             float64
dtype: object

## 함수파트

In [3]:
# QTY값 0보다 작은 값 체크 함수
def dataCheck(df):
    if df["QTY"] < 0:
        return True
    else:
        return False

In [4]:
# QTY값 0보다 작은 경우 0으로 수정 함수
def refining(df):
    if df["QTY"] < 0:
        return 0
    else:
        return df["QTY"]

In [5]:
# 과거주차 찾기 함수
def preWeek(inputYW, gapW):
    inpYW = str(inputYW) ## 입력 연도주차 문자열 변환
    inpY=inpYW[0:4] ## 입력 연도 추출
    inpW=inpYW[4: ] ## 입력 주차 추출

    inpGap=int(gapW) ## 입력 갭 인트형 확정

    dif = int(inpW) - inpGap ## 입력 갭과 입력 주차의 차

    if(dif>0):
        if(dif<10):
            lastanswer = str(inpY) + "0" + str(dif)
        else:
            lastanswer = str(inpY) + str(dif)
        
    else:
        dif = dif
        preyear = int(inpY)
        while (dif <= 0):
            preyear = preyear - 1
            preweek = Week.last_week_of_year(preyear).week
            dif = dif + preweek
        if(dif<10):
            lastanswer =str(preyear)+"0"+str(dif)
        else:
            lastanswer = str(preyear) + str(dif)

    return lastanswer

In [6]:
# 미래주차 찾기 함수
def postWeek(inputYearWeek, gapWeek):
    
    currYear = int(inputYearWeek[0:4])
    currWeek = int(inputYearWeek[4:])
    
    maxWeek = Week.last_week_of_year(currYear).week
    
    if (maxWeek < currWeek + gapWeek):
        iterGap = gapWeek + currWeek - maxWeek
        iterYear = currYear + 1
    
        iterWeek = Week.last_week_of_year(iterYear).week
        
        while (iterGap > 0):
            if (iterWeek < iterGap):
                iterGap = iterGap - iterWeek
                iterYear = iterYear + 1
                iterWeek = Week.last_week_of_year(iterYear).week
            else:
                iterWeek = iterGap
                iterGap = 0
        
        return str(iterYear) + "%02d" %(iterWeek)
    else:
        return str(currYear) + "%02d" %(currWeek + gapWeek)

In [7]:
# 주차 차이 계산 함수
def lagWeek(week1, week2):
    fromWeek = ""
    toWeek = ""

    if (int(week1) < int(week2)):
        fromWeek = week1
        toWeek = week2
    else:
        fromWeek = week2
        toWeek = week1

    # 53주차 제외
    currYear = int(fromWeek[0:4])
    currWeek = int(fromWeek[4:])

    if (52 < currWeek):
        return 0
    
    # 53주차 제외
    currYear = int(toWeek[0:4])
    currWeek = int(toWeek[4:])

    if (52 < currWeek):
        return 0

    calcGapNum = 0
    calcWeek = toWeek

    while (int(fromWeek) != int(calcWeek)):
        calcGapNum = calcGapNum + 1
        calcWeek = preWeek(toWeek, calcGapNum)
    return calcGapNum

## 2. 데이터 정제(부재 연주차 삽입)

In [8]:
# 2. 데이터 정제(부재 연주차 삽입)

# seasonality_dataData_df = seasonality_dataData_df.drop([0]) # 삭제 확인용
seasonality_dataData_df.reset_index()

minWeek = seasonality_dataData_df["YEARWEEK"].min()
currWeek = seasonality_dataData_df["YEARWEEK"].max()
gapWeek =lagWeek(str(minWeek),str(currWeek))

allWeekArray = []
startWeek = minWeek
endWeek = currWeek
i = 0

while(i <= gapWeek):
    allWeekArray.append(int(postWeek(str(startWeek), i)))
    i=i+1
    
allWeekset = set(allWeekArray)
# print("allWeekset 값: " , allWeekset)

groupby_fillWEEK = seasonality_dataData_df.groupby(["CONTINENT","REGIONID", "PRODUCTGROUP", "PRODUCT"])[["YEARWEEK"]]

data = {}
for columnName in seasonality_dataData.columns:
    data[columnName]= []

cnt = 0
# 그룹바이 전체를 가져온다.
for group in groupby_fillWEEK:
    access = 1 # 개별 그룹바이 결과 접근하기 위한 파라미터
    eachGroup = group[access].YEARWEEK
    weekArray=[]
    
    # 연주차 정보를 Set자료형으로 바꾼다. 차집합으로 비어있는 연주차 정보를 Set자료형으로 갖는다.
    for weekData in eachGroup:
        weekArray.append(weekData)
#     print("weekArray 값 : ", weekArray)
    
    weekSet= set(weekArray)
#     print("weekSet 값: " , weekSet)
    newWeekset = allWeekset.difference(weekSet)
#     print("newWeekset 값: ", newWeekset)
    # 그룹바이한 각각의 그룹에 대한 연주차 정보가 비어있지 않으면 그룹바이의 다음 그룹으로 넘어간다.
    if(len(newWeekset)==0):
        continue

    # 그룹바이한 데이터프레임의 값 접근하기 위한 파라미터
    CONTINENTcolNum = 0
    REGIONIDcolNum = 1
    PRODUCTGROUPcolNum = 2
    PRODUCTcolNum = 3
    
    
    # 개별 그룹바이 결과중에서 연주차 정보가 비어있는 그룹만 채워준다.
    for eachGroup in group:        
        if(newWeekset != None):
#             print(newWeekset)
            for insertRawData in newWeekset:
                data["CONTINENT"].append(eachGroup[CONTINENTcolNum])
                data["REGIONID"].append(eachGroup[REGIONIDcolNum])
                data["PRODUCTGROUP"].append(eachGroup[PRODUCTGROUPcolNum])
                data["PRODUCT"].append(eachGroup[PRODUCTcolNum])
                data["YEARWEEK"].append(insertRawData)
                data["QTY"].append(0)
                cnt=cnt+1
            break

# print("추가 행 개수: ", cnt)
# print(data)
# print(type(data))
# print("-------------------")
# print("-------------------")
addDf = pd.DataFrame(data=data)
# print(addDf)
# print("-------------------")
# print("-------------------")

concated_seasonality_dataData_df = pd.concat([addDf, seasonality_dataData_df])
# print(concated_seasonality_dataData_df)


## 2. 데이터 정제(QTY 음수값 0으로 보정, YEARWEEK(주차정보) 분리, 53주차 제외)

In [9]:
# 2. 데이터 정제(QTY 음수값 0으로 보정, YEARWEEK(주차정보) 분리, 53주차 제외)

# 이상데이터 확인 (0보다 작은 QTY의 개수 323개)
concated_seasonality_dataData_df["QTY_CHECK"] = concated_seasonality_dataData_df.apply(dataCheck, axis=1)
pd.value_counts(concated_seasonality_dataData_df.QTY_CHECK, sort=False)
#컬럼명.apply(pd.value_counts) 으로도 사용가능

# 이상데이터 정제 (0보다 작은 QTY를 0으로 변환)
concated_seasonality_dataData_df["QTY_REFINING"] = concated_seasonality_dataData_df.apply(refining, axis=1)

# YEARWEEK(주차정보) 분리
concated_seasonality_dataData_df["YEAR"] = concated_seasonality_dataData_df["YEARWEEK"].astype("str").str[0:4]
concated_seasonality_dataData_df["WEEK"] = concated_seasonality_dataData_df["YEARWEEK"].astype("str").str[4:6]
concated_seasonality_dataData_df["YEARWEEK"] = concated_seasonality_dataData_df["YEARWEEK"].astype("str")

# 53주차 제거
concated_seasonality_dataData_df = concated_seasonality_dataData_df[concated_seasonality_dataData_df.WEEK.astype("float") <= 52]
concated_seasonality_dataData_df.head()

,CONTINENT,PRODUCT,PRODUCTGROUP,QTY,REGIONID,YEARWEEK,QTY_CHECK,QTY_REFINING,YEAR,WEEK
0,N05,PRODUCT04,PG01,71.0,A60,201402.0,False,71.0,2014,02
1,N05,PRODUCT59,PG03,22275.0,A60,201402.0,False,22275.0,2014,02
2,N05,PRODUCT34,PG02,4463.0,A60,201402.0,False,4463.0,2014,02
3,N05,PRODUCT47,PG03,0.0,A60,201402.0,False,0.0,2014,02
4,N05,PRODUCT56,PG03,23.0,A60,201402.0,False,23.0,2014,02


## 3~5단계(계절성 지수 레벨, 이동평균 구하기, 상하한 및 스무딩 처리)

In [15]:
# 3. 그룹바이를 통한 계절성 지수 레벨 결정
sorted_seasonality_df= concated_seasonality_dataData_df.sort_values(["CONTINENT", "REGIONID", "PRODUCTGROUP", "PRODUCT", "YEARWEEK"], ascending=True)
# sorted_seasonality_df = sorted_seasonality_df.reset_index(drop=True)
group_seasonality_df = sorted_seasonality_df.groupby(["CONTINENT","REGIONID", "PRODUCTGROUP", "PRODUCT"])[["QTY"]]

# 4. 롤링을 통한 이동평균량 구하기(롤링함수 사용시 Rolling 값 보정)
groupRolling_seasonality_df_13MA= group_seasonality_df.rolling(window=13, min_periods=1,center=True).mean()
groupRolling_seasonality_df_13MA = groupRolling_seasonality_df_13MA.reset_index().set_index("level_4")

mergeData = pd.merge(sorted_seasonality_df, groupRolling_seasonality_df_13MA, left_index=True, right_index=True, suffixes=('', '_13MA'), how="left")

# print(groupRolling_seasonality_df_5MA.head(52))
# print(groupRolling_seasonality_df_5MA.astype)
# for i in groupRolling_seasonality_df_5MA["QTY"]:
#     print(i)

# result_seasonality_df["10MA"] = group_seasonality_df.rolling(window=10, center=True).mean()
# result_seasonality_df["15MA"] = group_seasonality_df.rolling(window=15, center=True).mean()
# result_seasonality_df["20MA"] = group_seasonality_df.rolling(window=20, center=True).mean()

# print(test.head())
# print(test.loc[298])

group_seasonality_df = sorted_seasonality_df.groupby(["CONTINENT","REGIONID", "PRODUCTGROUP", "PRODUCT"])[["QTY"]]
groupRolling_seasonality_df_5STD= group_seasonality_df.rolling(window=5, min_periods=1,center=True).std(ddof=1)
groupRolling_seasonality_df_5STD = groupRolling_seasonality_df_5STD.reset_index().set_index("level_4")

mergeData2 = pd.merge(mergeData, groupRolling_seasonality_df_5STD, left_index=True, right_index=True, suffixes=('', '_5STD'), how="left")

# 5. 상한선, 하한선, 스무딩 처리.
mergeData2["Cleaned_QTY"]= \
np.where(mergeData2["QTY"] > (mergeData2["QTY_13MA"]+mergeData2["QTY_5STD"]), (mergeData2["QTY_13MA"]+mergeData2["QTY_5STD"]),\
          np.where(mergeData2["QTY"] < (mergeData2["QTY_13MA"]-mergeData2["QTY_5STD"]), (mergeData2["QTY_13MA"]-mergeData2["QTY_5STD"]),\
            mergeData2["QTY"]))

mergeData2 = mergeData2[["CONTINENT","REGIONID", "PRODUCTGROUP", "PRODUCT", "YEAR", "WEEK", "QTY", "QTY_13MA", "QTY_5STD", "Cleaned_QTY"]]

group_seasonality_df = mergeData2.groupby(["CONTINENT","REGIONID", "PRODUCTGROUP", "PRODUCT"])[["Cleaned_QTY"]]
groupRolling_seasonality_smoothing_3MA = group_seasonality_df.rolling(window=3, min_periods=1,center=True).mean()
groupRolling_seasonality_smoothing_3MA
groupRolling_seasonality_smoothing_3MA = groupRolling_seasonality_smoothing_3MA.reset_index().set_index("level_4")
resultData = pd.merge(mergeData2, groupRolling_seasonality_smoothing_3MA, left_index=True, right_index=True, suffixes=('', '_SMTH'), how="left")


## 6. 실데이터 계절성, 스무딩처리 계절성

In [21]:
# 6. 실데이터 계절성, 스무딩처리 계절성
resultData = resultData[["CONTINENT","REGIONID", "PRODUCTGROUP", "PRODUCT", "YEAR", "WEEK", "QTY", "QTY_13MA", "QTY_5STD", "Cleaned_QTY", "Cleaned_QTY_SMTH"]]
# resultData.dtypes
resultData["SeasonalityScr_Actual"] = np.where(resultData["Cleaned_QTY_SMTH"]==0, 0, (resultData["QTY"] /resultData["Cleaned_QTY_SMTH"]))
resultData["SeasonalityScr_SMTH"] = np.where(resultData["Cleaned_QTY_SMTH"]==0, 0, (resultData["Cleaned_QTY"]/resultData["Cleaned_QTY_SMTH"]))

resultData

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,CONTINENT,REGIONID,PRODUCTGROUP,PRODUCT,YEAR,WEEK,QTY,QTY_13MA,QTY_5STD,Cleaned_QTY,Cleaned_QTY_SMTH,SeasonalityScr_Actual,SeasonalityScr_SMTH
298,N01,A00,PG02,PRODUCT34,2014,01,661.0,542.285714,53.873308,596.159022,592.774612,1.115095,1.005709
1757,N01,A00,PG02,PRODUCT34,2014,02,679.0,520.000000,69.390201,589.390201,587.849741,1.155057,1.002620
3125,N01,A00,PG02,PRODUCT34,2014,03,578.0,514.444444,73.984458,578.000000,566.463400,1.020366,1.020366
205,N01,A00,PG02,PRODUCT34,2014,04,532.0,516.800000,93.457477,532.000000,542.000000,0.981550,0.981550
4369,N01,A00,PG02,PRODUCT34,2014,05,516.0,503.363636,73.407765,516.000000,490.333333,1.052345,1.052345
4313,N01,A00,PG02,PRODUCT34,2014,06,423.0,490.750000,72.527926,423.000000,453.533524,0.932676,0.932676
6288,N01,A00,PG02,PRODUCT34,2014,07,407.0,480.230769,58.630197,421.600572,410.514996,0.991438,1.027004
6403,N01,A00,PG02,PRODUCT34,2014,08,364.0,453.384615,66.440199,386.944417,426.181663,0.854096,0.907933
5158,N01,A00,PG02,PRODUCT34,2014,09,470.0,423.846154,73.941193,470.000000,446.663611,1.052246,1.052246
7437,N01,A00,PG02,PRODUCT34,2014,10,538.0,401.230769,81.815646,483.046416,440.682139,1.220835,1.096133


## 7. ORACLE DB 저장 미구현